In [1]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq langchain==0.0.139 --progress-bar off
!pip install -qqq openai==0.27.4 --progress-bar off
!pip install -Uqqq watermark==2.3.1 --progress-bar off
!pip install -Uqqq chromadb==0.3.21 --progress-bar off
!pip install -Uqqq tiktoken==0.3.3 --progress-bar off

In [13]:
# This line of code is a magic command in IPython that loads the 'watermark' extension.
# The 'watermark' extension is a useful tool for documenting used Python modules, 
# their versions, and the machine characteristics. This can be helpful for reproducibility of computational environments.

%load_ext watermark

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark


In [14]:
# Import the necessary libraies
import os
import openai

import textwrap
import chromadb
import langchain
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import WebBaseLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Chroma

In [7]:
# Set the API key by reading the folder path. Use this code if you're running the code on Google Colab. Otherwise, use the actual folder path
folder_path = '/Users/jasper/Desktop/LangChain/'

# Folder path
os.chdir(folder_path)

In [10]:
# Read the text file containing the API key
with open(folder_path + "Jasper_OpenAI_API_Key.txt", "r") as f:
  openai.api_key = ' '.join(f.readlines())

In [11]:
# Update the OpenAI API key by updating the environment variable
os.environ["OPENAI_API_KEY"] = openai.api_key

In [15]:
def print_response(response: str):
    print("\n".join(textwrap.wrap(response, width=100)))

In [20]:
model = OpenAI(model = 'gpt-3.5-turbo-instruct', temperature=0)

WARNING! model is not default parameter.
                    model was transfered to model_kwargs.
                    Please confirm that model is what you intended.


In [21]:
print(
    model(
        "You're Dwight K. Schrute from the Office. Suggest 5 places to visit in Scranton that are connected to the TV show."
    )
)



1. Dunder Mifflin Paper Company - This is the fictional paper company where you work in the show. While there isn't an actual Dunder Mifflin office in Scranton, you can visit the building that was used as the exterior shot for the office in the show.

2. Poor Richard's Pub - This is the bar where you and your coworkers often go for drinks and karaoke. In real life, Poor Richard's is a popular bar and restaurant in Scranton that often hosts Office-themed events and trivia nights.

3. Steamtown National Historic Site - This train museum was featured in the episode "The Convention" where you and your coworkers attend a paper convention. You can visit the museum and see the train cars used in the show.

4. Lake Scranton - This is the lake where you and your coworkers go for a company picnic in the episode "The Merger." You can visit the lake and have a picnic of your own, just like in the show.

5. The Electric City sign - This iconic sign is featured in the opening credits of the show a